In [ ]:
# Extractig the training data
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.image import imread
import random
from PIL import Image

In [ ]:
data = pd.read_csv('../input/brain-tumor-curneu/Qn3_Braintumor/Brain Tumor.csv')

In [ ]:
data.head()

In [ ]:
data['path']='../input/brain-tumor-curneu/Qn3_Braintumor/Brain Tumor/'+data['Image']+'.jpg'

In [ ]:
data.describe()

In [ ]:
groups = np.array(data.Class.value_counts())
groups_cat = ['Positive', 'Negative']
plt.pie(groups, labels = groups_cat,startangle=90, autopct='%1.1f%%')
plt.axis('equal')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.image import imread
import random
fig = plt.figure(figsize=(10,10))
random.seed(15)
random_list=random.sample(range(0,1000),9)
for i,index in enumerate(random_list):
    plt.subplot(330+1+i)
    file_path=data.loc[index]['path']
    img=imread(file_path)
    plt.imshow(img)
plt.show()

In [ ]:
path=data['path'].tolist()
target=data['Class'].tolist()

In [ ]:
import random
random.seed(655)
def train_validation_split(path,target,split=0.7):
    train_y=[]
    val_x=[]
    val_y=[]
    train_x=random.sample(path,int(len(path)*split))
    for i in path:
        if i not in train_x:
            val_x.append(i)
            temp=path.index(i)
            val_y.append(target[temp])
        else:
            temp=path.index(i)
            train_y.append(target[temp])
    return train_x,val_x,train_y,val_y    

In [ ]:
train_x,val_x,train_y,val_y=train_validation_split(path,target)

In [ ]:
print('Training data:', len(train_x))
print('Training Label:', len(train_y))
print('Validation Data:',len(val_x))
print('Validation Label:', len(val_y))
print('------------------------------mpimg.imread(train_x[4])-------------')
print('Total Data:', len(train_x)+len(val_x))

In [ ]:
import matplotlib.image as mpimg
img = mpimg.imread(train_x[4])
print(img.shape)

In [ ]:
import cv2
from albumentations import (
    Compose, ToFloat
)

AUGMENTATIONS_TRAIN = Compose([
    ToFloat(max_value=255)
])

AUGMENTATIONS_TEST = Compose([
    # CLAHE(p=1.0, clip_limit=2.0),
    ToFloat(max_value=255)
])

In [ ]:
from tensorflow.keras.utils import Sequence
import cv2

In [ ]:
# for training data
class Brain_Tumor_Sequence(Sequence):
    def __init__(self, x_set, y_set, batch_size, augmentations, shape):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.augment = augmentations
        self.shape = shape

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        return np.stack([
            self.augment(image=mpimg.imread(x))["image"] for x in batch_x
        ], axis=0), np.array(batch_y)

In [ ]:
#for Testing data
class Brain_Tumor_Test_Sequence(Sequence):
    def __init__(self, x_set,batch_size, augmentations, shape):
        self.x = x_set
        self.batch_size = batch_size
        self.augment = augmentations
        self.shape = shape

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        #batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        return np.stack([
            self.augment(image=cv2.resize(cv2.imread(x, 1), self.shape))["image"] for x in batch_x
        ], axis=0)

In [ ]:
train_y=np.array(train_y).reshape((len(train_y),1))
val_y=np.array(val_y).reshape((len(val_y),1))

In [ ]:
train_gen=Brain_Tumor_Sequence(train_x,train_y,16, augmentations=AUGMENTATIONS_TRAIN,shape=(240,240))
val_gen=Brain_Tumor_Sequence(val_x,val_y,16, augmentations=AUGMENTATIONS_TRAIN,shape=(240,240))

In [ ]:
from tensorflow.keras.applications import InceptionResNetV2
model=InceptionResNetV2(include_top=False, weights='imagenet',input_shape=[240,240,3])

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras import Model
x=model.output

#x=Flatten(name='Flatten')(x)
x=GlobalAveragePooling2D(name='G_avg_pooling')(x)
'''x=Dense(1000,activation=('relu'),name="top_activation_2")(x)
x=Dropout(0.4, name="top_dropout2")(x)
'''
x=Dense(256,activation=('relu'), name="top_activation_3")(x)
x=Dropout(0.3, name="top_dropout3")(x)

x=Dense(1,activation="sigmoid", name="pred")(x)

final_model=Model(model.input,x)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Convolution2D,Activation,Flatten,Dense,Dropout,MaxPool2D,BatchNormalization
model = Sequential()

model.add(Conv2D(8, kernel_size=(3,3), padding='same', input_shape = (240,240,3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(3, 3)))

model.add(Conv2D(16, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [ ]:
from tensorflow.keras import losses, optimizers, callbacks
file_path='/kaggle/working/vgg.h5'
checkpoint= callbacks.ModelCheckpoint(file_path, monitor='val_loss',verbose=1,save_best_only=True)
model.compile(optimizer= optimizers.SGD(learning_rate=0.01),loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_gen,epochs=20,validation_data=val_gen,verbose=1,callbacks=[checkpoint])

In [ ]:
import numpy as np
plt.plot(history.history['loss'], color='b', label="Training loss")
plt.plot(history.history['val_loss'], color='r', label="validation loss")
plt.xticks(np.arange(1, 2, 1))
plt.yticks(np.arange(0, 1, 0.1))
plt.legend()
plt.title('Training Loss VS Validation Loss')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], color='b', label="Training accuracy")
plt.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
plt.xticks(np.arange(1, 2, 1))
plt.title('Training Accuracy VS Validation Accuracy')
plt.legend()
plt.show()